<a href="https://colab.research.google.com/github/AlanGPT-93/Amortization-Table-MONEX/blob/main/test_code_amortization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !python --version
! pip install version numpy_financial

# Inicio y Configuración

## Librerías

In [ ]:
## Carga de librerías
import pandas as pd
import functools
import numpy_financial as npf
import numpy as np
from dateutil.relativedelta import relativedelta
import datetime
import re

## Carga de Datos y Estandarización de Nombres

In [ ]:
# credits_table = pd.read_csv("Credits_data.csv", encoding= "latin")
credits_table = pd.read_excel("Credits_data.xlsx") ## BBDD de Ingresos

In [ ]:
gastos_table = pd.read_excel("Gastos_data.xlsx") ## BBDD de Gastos

In [ ]:
#tasas_historicas = pd.read_csv("Tasas_Hist.csv")
tasas_historicas = pd.read_excel("Tasas_Hist.xlsx") ## BBDD Tasas Históricas

In [ ]:
# CONSTANTES
BASE_DIAS_INTERES_ANUAL = 360.
DIAS_PROM_MES = 365 / 12.

# Diccionario para cambiar el nombre a las columnas de BBDD_INGRESOS
new_cols = {'Llave': 'ID',
'Producto': 'PRODUCTO',
'ID Crédito Asignado Inst': 'ID_CREDITO_ASIG',
'Sindicatos': 'SINDICATOS',
'No Cliente': 'NO_CLIENTE',
'Cliente': 'CLIENTE',
'Divisa Línea': 'DIVISA',
'Revolvente / No Rev Multiple / No Rev una sola': 'TIPO_CREDITO',
'Capital (traducción)': 'PAGO_CAPITAL',
'Interés (traducción)': 'PAGO_INTERESES',
'Tasa Ref 2': 'TASA_REFERENCIA',
'Máx. de Tasa Fija Valor': 'TASA_FIJA',
'Máx. de Monto Original línea': 'MONTO',
'Máx. de Plazo en Meses': 'PLAZO',
'Máx. de Período de Gracia Capital': 'PERIODO_GRACIA_CAP',
'Máx. de Período de Gracia Interés': 'PERIODO_GRACIA_INT',
'Máx. de Margen': 'TASA_MARGEN',
'Máx. de Fecha aprobación Comité': 'FECHA',
'Divisa la1': 'DIVISA_APERTURA',
'Monto la1': 'MONTO_APERTURA',
'%comisión por apertura': 'PCT_APERTURA',
'Divisa ld': 'DIVISA_DESEMBOLSO',
'Monto ld': 'MONTO_DESEMBOLSO',
'%comisión por desembolso': 'PCT_DESEMBOLSO',
'Divisa otras comisiones': 'DIVISA_OTRAS_COM',
'Monto otras comisiones': 'MONTO_OTRAS_COM',
'%otras comisiones': 'PCT_OTRAS_COM'}

new_cols_gastos = {'Descripción OC': 'AUX_ID',
'Fecha de creación': 'AUX_FECHA',
'Monto neto': 'MONTO_COSTO'
}

In [ ]:
credits_table.rename(columns = new_cols, inplace = True ) # Se cambia el nombre de las columnas de acuerdo al diccionario new_cols.

In [ ]:
gastos_table.rename(columns = new_cols_gastos, inplace = True ) # Se cambia el nombre de las columnas de acuerdo al diccionario new_cols_gastos.

## Configuración de Gastos

In [ ]:
## Se asigna el Tipo de gasto
com_type = ["Porcentaje del monto", "Fijo"]
sp_type = ["Incremento de Capital", "Sin Incremento de Capital"]

comision_apertura_tipo = com_type[0]
comision_disposicion_credito_tipo = com_type[0]
otras_comisiones_tipo = com_type[0]
prima_seguro_tipo = sp_type[0]
gastos_originacion_tipo = sp_type[1]
avaluos_tipo = sp_type[1]
gastos_investigacion_tipo = sp_type[1]
evaluacion_crediticia_tipo = sp_type[1]
evaluacion_garantias_tipo = sp_type[1]
negociaciones_terminos_tipo = sp_type[1]
v_gastos = np.array([prima_seguro_tipo,gastos_originacion_tipo, avaluos_tipo, gastos_investigacion_tipo, evaluacion_crediticia_tipo, evaluacion_garantias_tipo, negociaciones_terminos_tipo])

In [ ]:
## Frecuencia de pago
sp_freq_pago = ["Al inicio", "Anual", "Al inicio y anual"]

comision_apertura_freq_pago = sp_freq_pago[0]
comision_disposicion_credito_freq_pago = sp_freq_pago[0]
otras_comisiones_freq_pago = sp_freq_pago[0]
prima_seguro_freq_pago = sp_freq_pago[0]
gastos_originacion_freq_pago = sp_freq_pago[0]
avaluos_freq_pago = sp_freq_pago[0]
gastos_investigacion_freq_pago = sp_freq_pago[0]
evaluacion_crediticia_freq_pago = sp_freq_pago[0]
evaluacion_garantias_freq_pago = sp_freq_pago[0]
negociaciones_terminos_freq_pago = sp_freq_pago[0]

In [ ]:
## Costo de transaccion
cost_trans_type = ["No financiado","Financiado"]

comision_apertura_costo_transaccion = cost_trans_type[0]
comision_disposicion_credito_costo_transaccion = cost_trans_type[0]
otras_comisiones_costo_transaccion = cost_trans_type[0]
prima_seguro_costo_transaccion = cost_trans_type[1]
gastos_originacion_costo_transaccion = cost_trans_type[0]
avaluos_costo_transaccion = cost_trans_type[0]
gastos_investigacion_costo_transaccion = cost_trans_type[0]
evaluacion_crediticia_costo_transaccion = cost_trans_type[0]
evaluacion_garantias_costo_transaccion = cost_trans_type[0]
negociaciones_terminos_costo_transaccion = cost_trans_type[0]

In [ ]:
## Se asigna el Pagado por
paid_by = ["", "Cliente", "Monex"]

comision_apertura_pagado_por = paid_by[0]
comision_disposicion_credito_pagado_por = paid_by[0]
otras_comisiones_pagado_por = paid_by[0]
prima_seguro_pagado_por = paid_by[1]
gastos_originacion_pagado_por = paid_by[2]
avaluos_pagado_por = paid_by[2]
gastos_investigacion_pagado_por = paid_by[2]
evaluacion_crediticia_pagado_por = paid_by[2]
evaluacion_garantias_pagado_por = paid_by[2]
negociaciones_terminos_pagado_por = paid_by[2]

# Funciones

## Auxiliares de amortización


In [ ]:
def change_str_to_date(str_date, t = 1, format_ = "%d/%m/%Y"): # Función que permite pasar de str a date
    if format_ != "%d/%m/%Y": 
        str_date = datetime.datetime.strptime(str_date,"%m/%d/%Y")
    else:
        str_date = datetime.datetime.strptime(str_date,"%d/%m/%Y")
    
    if t == 1:
        return str_date.strftime("%d/%m/%Y")
    else:
        return str_date

In [ ]:
change_str_to_date_arg = functools.partial(change_str_to_date, t = 1, format_ = "%m/%d/%Y")
tasas_historicas["Tasa_Ref"] = list(map(change_str_to_date_arg,tasas_historicas["Tasa_Ref"]))

In [ ]:
# Se obtiene el valor numérico de la frecuencia de pago segun el insumo
def get_freq_val(freq_pago, plazo):
    if freq_pago in ["Mensual", "Int. x anticipado", "Otra Periodicidad"]: #== "Mensual":
        _ = 1
    elif freq_pago == "Bimestral":
        _ = 2
    elif freq_pago == "Trimestral":
        _ = 3
    elif freq_pago == "Semestral":
        _ = 6
    elif freq_pago == "Al vencimiento":
        _ = plazo
    else:
        _ = 0
    return _

In [ ]:
# Se obtiene la tasa de referencia segun el tipo.
def m_t_ref(i, tasa_referencia):
    if tasa_referencia == "fija":
        _ = credits_table.loc[i,"TASA_FIJA"] # Max_de_Tasa_Fija_Valor
    else:
        _ = credits_table.loc[i,"TASA_MARGEN"] # Max_de_Margen
    return _

In [ ]:
def get_parameters(credit_index): # Función que permite tener todos los parámetros de un crédito
    # Por credito
    i = credit_index#794 # indice de credito 1,,13 35, 56, 72, 50, 156, 582, 18, 476, 103
    linea_id = credits_table.loc[i,"ID"] # Identificador de la linea de credito, Llave
    freq_pago_capital =  credits_table.loc[i,"PAGO_CAPITAL"] # Frecuencia de pago de Capital Descripcion, Capital_traduccion
    freq_pago_intereses = credits_table.loc[i,"PAGO_INTERESES"] # Frecuencia de pago de Intereses Descripcion, Interes_traduccion
    tasa_referencia = credits_table.loc[i,"TASA_REFERENCIA"].strip() # Tasa de Referencia, Tasa_Ref_2
    margen_tasa_referencia = m_t_ref(i, tasa_referencia) # Margen Tasa de Referencia
    monto_original_linea = credits_table.loc[i,"MONTO"] # Monto original de la linea, Max_de_Monto_Original_linea
    plazo_original_linea = credits_table.loc[i,"PLAZO"] # Plazo Original de la linea en meses, Max_de_Plazo_en_Meses
    periodo_gracia_capital = credits_table.loc[i,"PERIODO_GRACIA_CAP"] # Periodo de gracia de Capital (Meses), Max_de_Periodo_de_Gracia_Capital
    periodo_gracia_intereses = credits_table.loc[i,"PERIODO_GRACIA_INT"] # Periodo de gracia de Intereses (Meses), Max_de_Periodo_de_Gracia_Interes
    divisa = credits_table.loc[i,"DIVISA"] # Divisa, Divisa_Linea
    fecha_inicio_credito = credits_table.loc[i,"FECHA"] # Max_de_Fecha_aprobacion_Comite
    fecha_inicio_credito_date = datetime.datetime.strptime(fecha_inicio_credito,"%m/%d/%Y") # "%d/%m/%Y" fecha_inicio_credito se convierte a datetime
    #fecha_fin_credito = datetime.date(2025, 1, 1)
    periodo_remanente_capital = plazo_original_linea - periodo_gracia_capital # Periodo remanente de Capital
    periodo_remanente_intereses = plazo_original_linea - periodo_gracia_intereses	# Periodo remanente de Intereses
    freq_pago_capital_val = get_freq_val(freq_pago_capital, plazo_original_linea)   # Frecuencia de pago de Capital (Meses)
    freq_pago_intereses_val	= get_freq_val(freq_pago_intereses, plazo_original_linea) # Frecuencia de pago de Intereses (Meses)

    mes = fecha_inicio_credito_date.month # mes de fecha de inicio
    anno = fecha_inicio_credito_date.year # anno de fecha de inicio
    fecha_inicio_credito_dia1 = datetime.date(anno,mes,1).strftime("%d/%m/%Y")
    date_row = tasas_historicas["Tasa_Ref"] == fecha_inicio_credito_dia1 # renglon de fecha Tasa_Ref
    t_referencia = float((tasas_historicas.loc[date_row, tasa_referencia].to_numpy()[0]).replace("%","")) #Tasa de referencia
    tasa = (t_referencia + float(margen_tasa_referencia)) / 100 # Tasa para calcular valores de interés.
    tasa_mensualizada = tasa * DIAS_PROM_MES / BASE_DIAS_INTERES_ANUAL

    return linea_id, freq_pago_capital, freq_pago_intereses, tasa_referencia, monto_original_linea,plazo_original_linea, periodo_gracia_capital,\
    periodo_gracia_intereses, periodo_remanente_capital, freq_pago_capital_val, freq_pago_intereses_val, tasa, tasa_mensualizada, anno, mes

In [ ]:
# Genera tabla en 0s con nombres de columnas segun el tipo de tabla, default es tabla de amortizacion.
def set_table(plazo_original_linea, anno, mes, table_type = "amortizacion"):
    fecha_inicio_credito_date_inicio = datetime.date(anno,mes,1)
    mes = list(range(0, plazo_original_linea + 1 ) ) # Lista que contiene los índices de los meses
    fecha_pago = [(fecha_inicio_credito_date_inicio + relativedelta(months = m)).strftime("%d/%m/%Y") for m in mes ] # Lista que contiene las fechas de inicio de mes
    mes_fecha = {"Mes": mes, "Fecha_Pago": fecha_pago} # Diccionario con las listas anteriores
    
    if table_type != "amortizacion":
        # Lista con el nombre de las columnas para tabla de costos
        table_col_names = ["Comision_por_apertura","Comision_por_disposicion","Otras_comisiones","Prima_del_seguro",\
                           "Costos_de_transaccion","Honorarios_comisiones_pagadas","Avaluos","Gastos_investigacion",\
                           "Evaluacion_crediticia_del_deudor","Evaluacion_reconocimiento_de_garantias",\
                           "Negociaciones_para_terminos_credito"]
    else:
        # Lista con el nombre de las columnas para tabla de amortizacion
        table_col_names = ["Monto_a_pagar","Interes_Ordinario_Previo","Interes_Ordinario","IVA", "Principal_Aux",\
                           "Principal","Saldo", "Aux_Int1","Aux_Int2","Aux_Principal1","Aux_Principal2","Aux_Principal3"]

    col_zeros = {acn : np.zeros( plazo_original_linea + 1 ) for acn in table_col_names } #Se inicializan valores de columnas en 0s
    table_ = pd.concat( [pd.DataFrame(mes_fecha), pd.DataFrame(col_zeros )] , axis= 1) # Se inicializa la Tabla de Amortizacion con
    # las columnas e indices necesarios.
    return table_

## Tabla de Amortización

In [ ]:
def create_amortization_table(parameters_):
    freq_pago_capital, freq_pago_intereses, tasa_referencia, monto_original_linea, plazo_original_linea, periodo_gracia_capital,\
    periodo_gracia_intereses, periodo_remanente_capital, freq_pago_capital_val, freq_pago_intereses_val, tasa, tasa_mensualizada, anno, mes = parameters_

    amortiazation_table = set_table(plazo_original_linea , anno, mes) # Inicia la tabla en 0s
    amortiazation_table.loc[0,"Saldo"] =  monto_original_linea
    s_interes = 0
    s_capital = 0

    for m in amortiazation_table.loc[1:,"Mes"] : # Se recorre por indice de mes
        """saldo_anterior = amortiazation_table.loc[m - 1 ,"Saldo"] # Se obtiene el saldo anterior

        # V. Se calcula columna Interes_Ordinario_Previo, se mantiende para todas las freq de pago.
        if m > plazo_original_linea: 
            amortiazation_table.loc[m ,"Interes_Ordinario_Previo"] = 0
        else:
            amortiazation_table.loc[m ,"Interes_Ordinario_Previo"] = (tasa * DIAS_PROM_MES / BASE_DIAS_INTERES_ANUAL) * saldo_anterior """

        # V2. Se calcula columna Aux_Int1, columna auxiliar para el calculo de columna Interes_Ordinario, se mantiende para todas las freq de pago..
        if m > plazo_original_linea:
            amortiazation_table.loc[m ,"Aux_Int1"] = 0
        elif m <= periodo_gracia_intereses:
            amortiazation_table.loc[m ,"Aux_Int1"] = 1
        else:
            try:
                amortiazation_table.loc[m ,"Aux_Int1"] = np.ceil( (m - periodo_gracia_intereses) / freq_pago_intereses_val )
            except:
                amortiazation_table.loc[m ,"Aux_Int1"] = 0

        # V2. Se calcula columna Aux_Int2, columna auxiliar para el calculo de columna Interes_Ordinario, se mantiende para todas las freq de pago..
        try:
            if m <= periodo_gracia_intereses or m > plazo_original_linea:
                amortiazation_table.loc[m ,"Aux_Int2"] = 0
            elif ((m - periodo_gracia_intereses) % freq_pago_intereses_val) == 0 or m == plazo_original_linea or m == periodo_gracia_intereses:
                amortiazation_table.loc[m ,"Aux_Int2"] = 1
            else:
                amortiazation_table.loc[m ,"Aux_Int2"] = 0
        except:
            amortiazation_table.loc[m ,"Aux_Int2"] = 0
        
        # V2. Se calcula columna Aux_Principal1, columna auxiliar para el calculo de columna Principal, se mantiende para todas las freq de pago.
        if m <= periodo_gracia_capital or m > plazo_original_linea:
            amortiazation_table.loc[m ,"Aux_Principal1"] = 0
        else:
            try:
                amortiazation_table.loc[m ,"Aux_Principal1"] = np.ceil( (m - periodo_gracia_capital) / freq_pago_capital_val )
            except:
                amortiazation_table.loc[m ,"Aux_Principal1"] = 0

        # V2. Se calcula columna Aux_Principal2, columna auxiliar para el calculo de columna Principal, se mantiende para todas las freq de pago.
        try:
            if m <= periodo_gracia_capital or m > plazo_original_linea:
                amortiazation_table.loc[m ,"Aux_Principal2"] = 0
            elif ((m - periodo_gracia_capital) % freq_pago_capital_val ) == 0 or m == plazo_original_linea:
                amortiazation_table.loc[m ,"Aux_Principal2"] = 1
            else:
                amortiazation_table.loc[m ,"Aux_Principal2"] = 0
        except:
            amortiazation_table.loc[m ,"Aux_Principal2"] = 0
        
        # V2. Se calcula columna Aux_Principal3, columna auxiliar para el calculo de columna Principal, se mantiende para todas las freq de pago.
        if periodo_gracia_capital == 0:
            amortiazation_table.loc[m ,"Aux_Principal3"] = amortiazation_table.loc[m ,"Aux_Principal2"]
        else:
            try:
                if m <= periodo_gracia_capital or m > plazo_original_linea:
                    amortiazation_table.loc[m ,"Aux_Principal3"] = 0
                elif ((m - periodo_gracia_capital + freq_pago_capital_val - 1 ) % freq_pago_capital_val ) == 0:
                    amortiazation_table.loc[m ,"Aux_Principal3"] = 1
                else:
                    amortiazation_table.loc[m ,"Aux_Principal3"] = 0
            except:
                amortiazation_table.loc[m ,"Aux_Principal3"] = 0


        if freq_pago_intereses != "Int. x anticipado":
            if m <= periodo_gracia_capital or m > plazo_original_linea: 
                    amortiazation_table.loc[m ,"Principal_Aux"]  = 0
            else:
                amortiazation_table.loc[m ,"Principal_Aux"]  = monto_original_linea / periodo_remanente_capital

    for m_ in amortiazation_table.loc[1:,"Mes"] :

        saldo_anterior = amortiazation_table.loc[m_ - 1 ,"Saldo"] # Se obtiene el saldo anterior

        # V2. Se calcula columna Interes_Ordinario_Previo, se mantiende para todas las freq de pago. Bien
        if m_ > plazo_original_linea: 
            amortiazation_table.loc[m_ ,"Interes_Ordinario_Previo"] = 0
        else:
            amortiazation_table.loc[m_ ,"Interes_Ordinario_Previo"] = (tasa * DIAS_PROM_MES / BASE_DIAS_INTERES_ANUAL) * saldo_anterior
            
        # V2. Se calcula columna Interes_Ordinario, usando Aux_Int1 y Aux_Int2, se mantiende para todas las freq de pago. Bien
        if freq_pago_intereses in ["Mensual", "Int. x anticipado", "Otra Periodicidad"]:  #== "Mensual":
            amortiazation_table.loc[m_ ,"Interes_Ordinario"] = amortiazation_table.loc[m_ ,"Interes_Ordinario_Previo"]
        elif freq_pago_intereses in ["Bimestral", "Trimestral", "Semestral", "Al vencimiento"] and amortiazation_table.loc[m_ ,"Aux_Int2"] == 1:
            s_interes += 1
            sum_index = np.where(amortiazation_table.loc[:,"Aux_Int1"] == s_interes) # Indice de perido
            amortiazation_table.loc[m_ ,"Interes_Ordinario"] = np.sum(amortiazation_table.loc[sum_index, "Interes_Ordinario_Previo"]) # Suma de intereses del periodo
        else:
            amortiazation_table.loc[m_ ,"Interes_Ordinario"] = 0

        
       ### Caso de Int. x anticipado
        if freq_pago_intereses == "Int. x anticipado":
            if freq_pago_capital == "Mensual":
                if m_ > plazo_original_linea:
                    amortiazation_table.loc[m_ ,"Monto_a_pagar"] = 0
                else:
                    amortiazation_table.loc[m_ ,"Monto_a_pagar"] = (monto_original_linea * tasa_mensualizada)/(1 - (1 + tasa_mensualizada)**(-plazo_original_linea) )
                
                amortiazation_table.loc[m_ ,"Principal_Aux"] =  amortiazation_table.loc[m_ ,"Monto_a_pagar"] - amortiazation_table.loc[m_ ,"Interes_Ordinario_Previo"]
                amortiazation_table.loc[m_ ,"Principal"] = amortiazation_table.loc[m_ ,"Principal_Aux"]
            
            else:
                if m_ <= periodo_gracia_capital or m_ > plazo_original_linea:
                    amortiazation_table.loc[m_ ,"Principal_Aux"] = 0
                else:
                    amortiazation_table.loc[m_ ,"Principal_Aux"] = monto_original_linea / periodo_remanente_capital
                
                if amortiazation_table.loc[m_ ,"Aux_Principal3"] == 1:
                    s_capital += 1
                    capital_sum_index = np.where(amortiazation_table.loc[:,"Aux_Principal1"] == s_capital) # Indice de perido
                    amortiazation_table.loc[m_ ,"Principal"] = np.sum(amortiazation_table.loc[capital_sum_index, "Principal_Aux"]) # Suma de intereses del periodo
                else:
                    amortiazation_table.loc[m_ ,"Principal"] = 0
                
                amortiazation_table.loc[m_ ,"Monto_a_pagar"] = amortiazation_table.loc[m_ ,"Interes_Ordinario"] + amortiazation_table.loc[m_ ,"IVA"] + amortiazation_table.loc[m_ ,"Principal"]
        
        ### Caso de "Bimestral", "Trimestral", "Semestral", "Al vencimiento"
        else:
            # V2. Se calcula columna Principal_Aux, se mantiende para todas las freq de pago.
            #if m_ <= periodo_gracia_capital or m_ > plazo_original_linea: 
            #    amortiazation_table.loc[m_ ,"Principal_Aux"]  = 0
            #else:
            #    amortiazation_table.loc[m_ ,"Principal_Aux"]  = monto_original_linea / periodo_remanente_capital

            # V2. Se calcula columna Principal, usando Principal_Aux, Aux_Principal1 y Aux_Principal3
            if freq_pago_capital == "Mensual":
                amortiazation_table.loc[m_ ,"Principal"] = amortiazation_table.loc[m_ ,"Principal_Aux"]
            elif freq_pago_capital in ["Bimestral", "Trimestral", "Semestral", "Al vencimiento"] and amortiazation_table.loc[m_ ,"Aux_Principal3"] == 1:
                s_capital += 1
                capital_sum_index = np.where(amortiazation_table.loc[:,"Aux_Principal1"] == s_capital) # Indice de perido
                amortiazation_table.loc[m_ ,"Principal"] = np.sum(amortiazation_table.loc[capital_sum_index, "Principal_Aux"]) # Suma de intereses del periodo
            else:
                amortiazation_table.loc[m_ ,"Principal"] = 0
            
            # V2. Se calcula columna Monto a Pagar, se mantiende para todas las freq de pago.
            if freq_pago_intereses == "Int. x anticipado":
                if freq_pago_capital == "Al vencimiento":
                    amortiazation_table.loc[m_ ,"Monto_a_pagar"] = amortiazation_table.loc[m_ ,"Interes_Ordinario"] + amortiazation_table.loc[m_ ,"IVA"] + amortiazation_table.loc[m_ ,"Principal"]
                elif m_ > freq_pago_capital_val:
                    0
                else:
                    amortiazation_table.loc[m_ ,"Monto_a_pagar"] = (monto_original_linea * tasa_mensualizada)/(1 - (1 + tasa_mensualizada)**(-plazo_original_linea) )
            else:
                amortiazation_table.loc[m_ ,"Monto_a_pagar"] = amortiazation_table.loc[m_ ,"Interes_Ordinario"] + amortiazation_table.loc[m_ ,"IVA"] + amortiazation_table.loc[m_ ,"Principal"]
            
        # V2. Se calcula columna Saldo, se mantiende para todas las freq de pago.
        amortiazation_table.loc[m_ ,"Saldo"] = saldo_anterior - amortiazation_table.loc[m_ ,"Principal"]
    
    
    return amortiazation_table

## Auxiliares de Tabla de Gastos

In [ ]:
def get_split_array(column_): # Función que divide texto en una lista
    aux_array = []
    for aux_ in column_:
        aux_array.append(aux_.split())
     
    return np.array(aux_array)

In [ ]:
def get_fst_day(date_):
    mes = date_.month # mes de fecha de inicio
    anno = date_.year # anno de fecha de inicio
    return datetime.date(anno,mes,1).strftime("%d/%m/%Y")

In [ ]:
aux_id_array = get_split_array(gastos_table["AUX_ID"]) # Se divide en 2 columnas AUX_ID
gastos_table["tipo_gasto2"] = aux_id_array[:,0] # Primera parte se asigna a tipo_gasto2
gastos_table["Llave2"] = aux_id_array[:,1] # Segunda parte se asigna a Llave2

In [ ]:
aux_fecha = get_split_array(gastos_table["AUX_FECHA"]) # Se toma la columna AUX_FECHA y se divide
aux_fecha = aux_fecha[:,0] 
change_str_to_date_arg = functools.partial(change_str_to_date, t = 0, format_ = "%d/%m/%Y") #Se transforma en fecha con formato dd/mm/yyyy
aux_fecha = list(map(get_fst_day, list(map(change_str_to_date_arg,aux_fecha) ))) #Se obtiene fecha al primer día del mes
gastos_table["Anio_mes2"] = aux_fecha # El resultado se asigna a Anio_mes2

In [ ]:
gastos_table["Gasto2"] = gastos_table["Total OC"] #Se asigna el valor de Gasto2 a Total OC

In [ ]:
# Esta función ayuda a calcular valores de las columnas de: Prima de seguro,
#honoraios, avalus, gastos de investigación, evaluación crediticia, etc.
def small_conditionals(m_index, plazo_ol, var_freq_pago, var_valor):
    alt_val = 0
    if m_index > plazo_ol:
        alt_val = 0
    elif var_freq_pago == "Al inicio" and m_index == 0:
        alt_val = var_valor
    elif var_freq_pago == "Anual" and m_index % 12 == 0 and m != 0:
        alt_val = var_valor
    elif var_freq_pago == "Al inicio y anual" and m_index % 12 == 0:
        alt_val = var_valor
    else:
        alt_val = 0
    return alt_val

In [ ]:
# Ayuda a calular columnas Comisión por apertura, Comisión por disposición,
# Otras comisiones
def big_conditionals(m_index, plazo_ol, var_freq_pago, var_tipo, var_valor, monto_original_linea):
    alt_val = 0
    if m_index > plazo_ol:
        alt_val = 0
    elif var_freq_pago == "Al inicio" and m_index == 0:
        if var_tipo == "Porcentaje del monto":
            alt_val = monto_original_linea * var_valor
        elif var_tipo == "Fijo":
            alt_val = var_valor
        else:
            alt_val = 0

    elif var_freq_pago == "Anual" and m_index % 12 == 0 and m_index != 0:
        if var_tipo == "Porcentaje del monto":
            alt_val = monto_original_linea * var_valor
        elif var_tipo == "Fijo":
            alt_val = var_valor
        else:
            alt_val = 0

    elif var_freq_pago == "Al inicio y anual" and m_index % 12 == 0:
        if var_tipo == "Porcentaje del monto":
            alt_val = monto_original_linea * var_valor
        elif var_tipo == "Fijo":
            alt_val = var_valor
        else:
            alt_val = 0
    else:
        alt_val = 0

    return alt_val

## Tabla de Gastos

In [ ]:
# Función principal que ayuda a calcular la tabla de gastos.
def create_cost_table(monto_original_linea, plazo_original_linea, anno, mes ):
    cost_table = set_table(plazo_original_linea, anno, mes ,"costos")
    for m in cost_table["Mes"]:
        # Big conditionals
        cost_table.loc[m, "Comision_por_apertura"] = big_conditionals(m, plazo_original_linea, comision_apertura_freq_pago, comision_apertura_tipo, comision_apertura_valor, monto_original_linea)
        cost_table.loc[m, "Comision_por_disposicion"] = big_conditionals(m, plazo_original_linea, comision_disposicion_credito_freq_pago, comision_disposicion_credito_tipo, comision_disposicion_credito_valor, monto_original_linea)
        cost_table.loc[m, "Otras_comisiones"] = big_conditionals(m, plazo_original_linea, otras_comisiones_freq_pago, otras_comisiones_tipo ,otras_comisiones_valor, monto_original_linea)
        #print(m)
        # Small conditonals
        cost_table.loc[m, "Prima_del_seguro"] = small_conditionals(m, plazo_original_linea, prima_seguro_freq_pago, prima_seguro_valor)
        cost_table.loc[m, "Honorarios_comisiones_pagadas"] = small_conditionals(m, plazo_original_linea, gastos_originacion_freq_pago, gastos_originacion_valor)
        cost_table.loc[m, "Avaluos"] = small_conditionals(m, plazo_original_linea, avaluos_freq_pago, avaluos_valor)
        cost_table.loc[m, "Gastos_investigacion"] = small_conditionals(m, plazo_original_linea, gastos_investigacion_freq_pago, gastos_investigacion_valor)
        cost_table.loc[m, "Evaluacion_crediticia_del_deudor"] = small_conditionals(m, plazo_original_linea, evaluacion_crediticia_freq_pago, evaluacion_crediticia_valor)
        cost_table.loc[m, "Evaluacion_reconocimiento_de_garantias"] = small_conditionals(m, plazo_original_linea, evaluacion_garantias_freq_pago, evaluacion_garantias_valor)
        cost_table.loc[m, "Negociaciones_para_terminos_credito"] = small_conditionals(m, plazo_original_linea, negociaciones_terminos_freq_pago, negociaciones_terminos_valor)
    cost_table["Costos_de_transaccion"] = cost_table["Honorarios_comisiones_pagadas"] + cost_table["Avaluos"] + cost_table["Gastos_investigacion"] + cost_table["Evaluacion_crediticia_del_deudor"] + cost_table["Evaluacion_reconocimiento_de_garantias"] + cost_table["Negociaciones_para_terminos_credito"]
    return cost_table

In [ ]:
#sum(cost_table["Prima_del_seguro"]), sum(cost_table["Costos_de_transaccion"]), sum(cost_table["Avaluos"])

## Tabla Amortización Desglose

In [ ]:
def create_new_amortization_table( freq_pago_int, plazo_or_linea, tie_mensual, cost_totals ):

    new_amort_table = amortization_table.copy()
    s_interes = 0

    ## Ingresos
    new_amort_table["n_Int_Ord_Prev"] = 0.
    new_amort_table["n_Principal"] = 0.
    new_amort_table["n_Saldo"] = 0.
    new_amort_table["n_Int_Ord"] = 0.
    new_amort_table["n_Diff_amortizacion"] = 0.

    ## Gastos
    new_amort_table["n_Comision_por_apertura"] = 0.
    new_amort_table["n_Comision_por_disposicion"] = 0.
    new_amort_table["n_Otras_comisiones"] = 0.
    new_amort_table["n_Prima_del_seguro"] = 0.
    new_amort_table["n_Costos_de_transaccion"] = 0.
    new_amort_table["n_Honorarios_comisiones_pagadas"] = 0.
    new_amort_table["n_Avaluos"] = 0.
    new_amort_table["n_Gastos_investigacion"] = 0.
    new_amort_table["n_Evaluacion_crediticia_del_deudor"] = 0.
    new_amort_table["n_Evaluacion_reconocimiento_de_garantias"] = 0.
    new_amort_table["n_Negociaciones_para_terminos_credito"] = 0.

    new_amort_table.loc[0,"n_Saldo"] =  -1 * new_amort_table.loc[0,"Flujo_estimado"]

    for m in new_amort_table.loc[1:,"Mes"]:
        
        ## Nuevo Int Ord Previo
        if m > plazo_or_linea:
            new_amort_table.loc[m, "n_Int_Ord_Prev"] = 0
        else:
            new_amort_table.loc[m, "n_Int_Ord_Prev"] = tie_mensual *  new_amort_table.loc[m -1 ,"n_Saldo"]
        
        ## Nuevo  Principal
        new_amort_table.loc[m, "n_Principal"] = new_amort_table.loc[m, "Flujo_estimado"] - new_amort_table.loc[m, "n_Int_Ord_Prev"]

        ## Nuevo Saldo
        new_amort_table.loc[m, "n_Saldo"] = new_amort_table.loc[ m - 1, "n_Saldo"] - new_amort_table.loc[m, "n_Principal"]

        ## Nuevo Int Ord
        if freq_pago_int == "Mensual":
            new_amort_table.loc[m, "n_Int_Ord"] = new_amort_table.loc[m, "n_Int_Ord_Prev"]
        elif freq_pago_int in ["Bimestral", "Trimestral", "Semestral", "Al vencimiento"] and new_amort_table.loc[m ,"Aux_Int2"] == 1:
            s_interes += 1
            sum_index = np.where(new_amort_table.loc[:,"Aux_Int1"] == s_interes) # Indice de perido
            new_amort_table.loc[m ,"n_Int_Ord"] = np.sum(new_amort_table.loc[sum_index, "n_Int_Ord_Prev"]) # Suma de intereses del periodo
        else:
            new_amort_table.loc[m ,"n_Int_Ord"] = 0

        ## Diff Amortizacion
        new_amort_table.loc[m ,"n_Diff_amortizacion"] = new_amort_table.loc[m ,"Interes_Ordinario"] - new_amort_table.loc[m ,"n_Int_Ord"]
    
    s_diff = np.sum(new_amort_table["n_Diff_amortizacion"])

    for m in new_amort_table.loc[1:,"Mes"]:
        new_amort_table.loc[m, "n_Comision_por_apertura"] = (cost_totals["Comision_por_apertura"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Comision_por_disposicion"] = (cost_totals["Comision_por_disposicion"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Otras_comisiones"] = (cost_totals["Otras_comisiones"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Prima_del_seguro"] = (cost_totals["Prima_del_seguro"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Costos_de_transaccion"] = (cost_totals["Costos_de_transaccion"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Honorarios_comisiones_pagadas"] = (cost_totals["Honorarios_comisiones_pagadas"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Avaluos"] = (cost_totals["Avaluos"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Gastos_investigacion"] = (cost_totals["Gastos_investigacion"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Evaluacion_crediticia_del_deudor"] = (cost_totals["Evaluacion_crediticia_del_deudor"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Evaluacion_reconocimiento_de_garantias"] = (cost_totals["Evaluacion_reconocimiento_de_garantias"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
        new_amort_table.loc[m, "n_Negociaciones_para_terminos_credito"] = (cost_totals["Negociaciones_para_terminos_credito"] / s_diff) * new_amort_table.loc[m ,"n_Diff_amortizacion"]
    
    return new_amort_table


In [ ]:
#new_amort_table = create_new_amortization_table( params[1], params[4], tie_mensual, cost_table_totals )

In [ ]:
#new_amort_table.head()

## Tabla Output

In [ ]:
## Tabla de Fechas, comieza desde 2014 
fecha_inicio_credits = datetime.date(2014,1,1)
mes_inicio_credits = list(range(0, 450 ) ) # Lista que contiene los índices de los meses
fecha_creditos_pago = [(fecha_inicio_credits + relativedelta(months = m)).strftime("%d/%m/%Y") for m in mes_inicio_credits ] # Lista que contiene las fechas de inicio de mes
mes_fechas_creditos = {"Fecha_Pago": fecha_creditos_pago} # Diccionario con las fechas desde ene-14
mes_fechas_creditos = pd.DataFrame(mes_fechas_creditos) # se crea DF

In [ ]:
def final_table():
    mes_fechas_creditos2 = mes_fechas_creditos.copy() # Se genera Copia de fehas
    df_test = new_amort_table.loc[:,['Fecha_Pago','n_Comision_por_apertura', 'n_Comision_por_disposicion', 'n_Otras_comisiones','n_Prima_del_seguro', 'n_Costos_de_transaccion']].copy() # Se toman columnas de la nueva tabla de amortizacion
    
    gastos_originacion_sum_index_admon =  np.where( (gastos_table["Llave2"] == linea_id) & (gastos_table["tipo_gasto2"] == "ADMINISTRACIÓN")) #Se obtienen los gastos de originacion
    gastos_originacion_valor_admon = np.sum(gastos_table.loc[gastos_originacion_sum_index_admon, "Gasto2"]) #se suma el total de gastos de originacion de acuerdo al id de credito
    df_gastos_admon = gastos_table.loc[gastos_originacion_sum_index_admon[0], ["Anio_mes2","Gasto2"]].groupby(["Anio_mes2"]).agg({"Gasto2":"sum"}) # Se realiza la suma por mes.


    mes_fechas_creditos2 = mes_fechas_creditos2.merge(df_test, how= "left", on="Fecha_Pago" ) # Se realiza cruce de tabla df_test y tabla de fechas
    mes_fechas_creditos2 = mes_fechas_creditos2.merge(df_gastos_admon, how= "left", left_on="Fecha_Pago", right_on= "Anio_mes2" ) # Se realiza cruce de tabla de gastos de admin y tabla anterior.
    mes_fechas_creditos2.fillna(-0, inplace= True) # Se llena con 0s los NAs
    mes_fechas_creditos2 = mes_fechas_creditos2.set_index("Fecha_Pago").T # se transpone matriz
    mes_fechas_creditos2.reset_index(inplace=True) #Se resetean los indices
    mes_fechas_creditos2["index"] = ["Comision por apertura", "Comision por disposicion", "Otras comisiones", "Prima del seguro",\
                                "Gastos de originacion", "Gastos de administracion"]

    df_aux_credit_values = credits_table["ID"] == linea_id #Se obtienen datos de la tabla de creditos segun id de credito
    df_aux_credit_values = credits_table.loc[df_aux_credit_values, ["FECHA", "ID", "PRODUCTO", "CLIENTE", "TIPO_CREDITO", "PAGO_CAPITAL", "PAGO_INTERESES",\
                                                                    "MONTO", "TASA_REFERENCIA","TASA_FIJA","TASA_MARGEN","PLAZO"]]

    newdf = pd.DataFrame(np.repeat(np.append(df_aux_credit_values.values, [[params[-4] , monto_neto_financiado, tie_anual]], axis =1), 6, axis=0),\
                        columns = list(df_aux_credit_values.columns) + ["TASA","MONTO_NET_FINANCIADO", "TASA_INTERES_EFECTIVA"]) # Se genera un nuevo DF, con valores generados en el proceso
    
    newdf["T=0"] = np.append(cost_table.loc[0,["Comision_por_apertura","Comision_por_disposicion","Otras_comisiones","Prima_del_seguro",]].values,\
          [-cost_table.loc[0,["Costos_de_transaccion"]][0], -gastos_originacion_valor_admon] )
    
    mes_fechas_creditos2 = pd.concat( [newdf, mes_fechas_creditos2], axis= 1 ) # Se agregan las tablas de newdf y mes_fechas_creditos2

    return mes_fechas_creditos2

# Ejecución de Proceso

In [ ]:
credit_idex_list = list(credits_table.index)

In [ ]:
table_f_append = pd.DataFrame()

In [ ]:
for cil in credit_idex_list:

    params = get_parameters(cil) # 381, 1099, 1027, 1175, 469, 1095, 535, 284, 393,168,948, 678 , 850
    linea_id = params[0]
    params = list(params[1:])

    #Se filtra la tabla de gatstos con el valor de linea_id
    linea_id_index = credits_table["ID"] == linea_id

    ## Se asigna el Valor
    comision_apertura_valor = float(credits_table.loc[linea_id_index,"PCT_APERTURA"]) #0
    comision_disposicion_credito_valor = float(credits_table.loc[linea_id_index,"PCT_DESEMBOLSO"]) #0
    otras_comisiones_valor = float(credits_table.loc[linea_id_index,"PCT_OTRAS_COM"]) #0
    prima_seguro_valor = 0
    gastos_originacion_sum_index =  np.where( (gastos_table["Llave2"] == linea_id) & (gastos_table["tipo_gasto2"] == "ORIGINACIÓN"))
    gastos_originacion_valor = np.sum(gastos_table.loc[gastos_originacion_sum_index, "Gasto2"])
    avaluos_valor = 0
    gastos_investigacion_valor = 0
    evaluacion_crediticia_valor = 0
    evaluacion_garantias_valor = 0
    negociaciones_terminos_valor = 0

    i_v_gastos = np.where(v_gastos == "Incremento de Capital")
    v_gastos_valor = np.array([prima_seguro_valor,gastos_originacion_valor, avaluos_valor, gastos_investigacion_valor, evaluacion_crediticia_valor, evaluacion_garantias_valor, negociaciones_terminos_valor])

    s_gastos_valor = np.sum(v_gastos_valor[i_v_gastos] )

    params[3] = params[3] + s_gastos_valor

    # Se crea Tabla de Amortizacion
    amortization_table = create_amortization_table(params)

    ## Calulo de monto neto Financiado
    cost_table = create_cost_table(params[3], params[4],params[-2],params[-1]) # Se crea Tabla de Costos
    cost_table_totals = cost_table.iloc[:,2:].sum() # Se realiza la suma de las columnas de la tabla de costos, excepto las primeras 2
    cost_table_totals = cost_table_totals * np.array([-1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1]) # Se realiza la multiplicación según signo.

    monto_neto_financiado = params[3] + np.sum([cost_table_totals]) - cost_table_totals["Costos_de_transaccion"] # Monto Neto Financiado. 
    #Se realiza la suma de Capital Total + Gastos
    amortization_table["Flujo_estimado"] = amortization_table["Interes_Ordinario"] + amortization_table["Principal"]
    amortization_table.loc[0,"Flujo_estimado"] = monto_neto_financiado * -1

    flujos_estimados_recibir = np.sum( amortization_table["Flujo_estimado"][1:]) #Suma de columna de Flujo Estimado
    sum_intereses_efectivo = flujos_estimados_recibir - monto_neto_financiado # Calculo de interes efectivo

    tie_mensual = round(npf.irr(amortization_table["Flujo_estimado"]) , 9) # Se realiza calculo de Tasa de Interes Efectiva de acuerdo a Flujo Estimado
    tie_anual = round(tie_mensual * BASE_DIAS_INTERES_ANUAL / DIAS_PROM_MES, 8) # TIE anual

    new_amort_table = create_new_amortization_table( params[1], params[4], tie_mensual, cost_table_totals ) # Tabla desglose

    table_f_append = table_f_append.append(final_table()) # Se adjunta tabla output al resto para generar una sola tabla

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
table_f_append.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7164 entries, 0 to 5
Data columns (total 52 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FECHA                  7164 non-null   object 
 1   ID                     7164 non-null   object 
 2   PRODUCTO               7164 non-null   object 
 3   CLIENTE                7164 non-null   object 
 4   TIPO_CREDITO           7164 non-null   object 
 5   PAGO_CAPITAL           7164 non-null   object 
 6   PAGO_INTERESES         7164 non-null   object 
 7   MONTO                  7164 non-null   object 
 8   TASA_REFERENCIA        7164 non-null   object 
 9   TASA_FIJA              966 non-null    object 
 10  TASA_MARGEN            7164 non-null   object 
 11  PLAZO                  7164 non-null   object 
 12  TASA                   7164 non-null   object 
 13  MONTO_NET_FINANCIADO   7164 non-null   object 
 14  TASA_INTERES_EFECTIVA  7164 non-null   object 
 15  T=0    